In [12]:
from PIL import Image
import os
path = r"D:/Face_Yang Che-Min/images/"
name = os.listdir(path)
print(name)
print(len(name))

['N01.jpg', 'N02.jpg', 'N03.jpg', 'N04.jpg', 'N05.jpg', 'N06.jpg', 'N07.jpg', 'N08.jpg', 'N09.jpg', 'N10.jpg', 'N11.jpg', 'N12.jpg', 'N14.jpg', 'N15.jpg', 'N16.jpg', 'N17.jpg', 'N18.jpg', 'N19.jpg', 'N20.jpg', 'N21.jpg', 'N22.jpg', 'N23.jpg', 'N24.jpg', 'N25.jpg', 'N26.jpg', 'N27.jpg', 'N28.jpg', 'N29.jpg', 'N30.jpg', 'Q1.jpg', 'Q10.jpg', 'Q11.jpg', 'Q13.jpg', 'Q14.jpg', 'Q15.jpg', 'Q16.jpg', 'Q17.jpg', 'Q18.jpg', 'Q19.jpg', 'Q2.jpg', 'Q20.jpg', 'Q21.jpg', 'Q22.jpg', 'Q23.jpg', 'Q24.jpg', 'Q25.jpg', 'Q26.jpg', 'Q27.jpg', 'Q28.jpg', 'Q29.jpg', 'Q30.jpg', 'Q31.jpg', 'Q32.jpg', 'Q34.jpg', 'Q35.jpg', 'Q36.jpg', 'Q37.jpg', 'Q4.jpg', 'Q5.jpg', 'Q6.jpg', 'Q8.jpg', 'Q9.jpg', 'Y01.jpg', 'Y02.jpg', 'Y03.jpg', 'Y04.jpg', 'Y05.jpg', 'Y06.jpg', 'Y07.jpg', 'Y08.jpg', 'Y09.jpg', 'Y10.jpg', 'Y11.jpg', 'Y12.jpg', 'Y13.jpg', 'Y14.jpg', 'Y15.jpg', 'Y16.jpg', 'Y17.jpg', 'Y18.jpg', 'Y19.jpg', 'Y20.jpg', 'Y21.jpg', 'Y22.jpg', 'Y23.jpg', 'Y24.jpg', 'Y25.jpg', 'Y26.jpg', 'Y27.jpg', 'Y28.jpg', 'Y29.jpg', 'Y30.

In [13]:
for i in range(len(name)):
    img = Image.open('D:/Face_Yang Che-Min/images/'+'%s'%(name[i]))
    img.save('D:/Face_Yang Che-Min/%s.png' %(name[i][:3]))